スクリプト実行に必要な環境をセットアップします。

In [ ]:
%run startup.py
load_questions()

In [3]:
import os
os.system('python startup.py')
  
# Initialize a network and a snapshot
bf_set_network("network-example-filters")
SNAPSHOT_NAME = "mycurrent"
SNAPSHOT_PATH = "networks/example-filters/mycurrent"
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)
load_questions()

parse_status = bfq.fileParseStatus().answer().frame()
parse_status

Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf_upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf_upload_diagnostics)


,File_Name,Status,Nodes
0,configs/N9k-v1.cfg,PASSED,['n9kv-1']
1,configs/after_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['after_summary_asa']
2,configs/before_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['before_summary_asa']
3,configs/ciscoasa.cfg,PARTIALLY_UNRECOGNIZED,['ciscoasa']


In [4]:
import os
import sys
import re
#textfsmのパスをnotebookに渡す
sys.path.append('/Users/lisago/python/ansible/venv/lib/python3.7/site-packages')

import textfsm

#acl_file = 'networks/example-filters/mycandidate/config/ASA_acl.cfg'
#ntc_template = 'ntc-templates/cisco_asa_show_access-list.template'
#acl_file = 'networks/example-filters/mycandidate/config/nxos_acl_sample.cfg'
#ntc_template = 'ntc-templates/cisco_nxos_show_access-lists.template'
interface_config_file = 'networks/example-filters/mycandidate/config/cisco_ios_show_ip_interfaces.cfg'
ntc_template = 'ntc-templates/cisco_ios_show_ip_interfaces.template'

'''
[{'INTF': 'Serial0', 'LINK_STATUS': 'down', 'PROTOCOL_STATUS': 'down', 'IPADDR': ['192.168.1.1/24*'],
'MASK': ['1'], 'VRF': '', 'MTU': '1500', 'IP_HELPER': [], 'OUTGOING_ACL': '', 'INBOUND_ACL': ''}]
'''

def output(interface, template):
    with open(interface, 'r') as f:
        inventory_text = f.read()
    
    with open(ntc_template) as f:
        #table = textfsm.TextFSM(f)
        fsm = textfsm.TextFSM(f)
        fsm_results = fsm.ParseText(inventory_text)
        parsed = [dict(zip(fsm.header, row)) for row in fsm_results]
        f.close()
    return parsed

def extract_interface(parsed):
    address_list = []
    for con in parsed:
        ipaddress = con['IPADDR']
        for add in ipaddress:
            #address.strip('*')
            address = re.sub(r'\*', ' ', add)
            address_list.append(address)
            
    return address_list
    

parsed = output(interface_config_file, ntc_template)
address_list = extract_interface(parsed)

for address in address_list:
    ip_flow = HeaderConstraints(srcIps=address)
                             #dstIps='255.255.255.0')
    answer = bfq.testFilters(headers=ip_flow,
                             nodes='before_summary_asa',
                             filters="SPLIT-ACL").answer() 
    show(answer.frame())
    


    

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,before_summary_asa,SPLIT-ACL,Start Location: before_summary_asaSrc IP: 192.168.1.0Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,PERMIT,permit 10.20.0.0 255.255.255.0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by standard ipv4 access-list named SPLIT-ACL, index 0: permit 10.20.0.0 255.255.255.0', lineDescription='permit 10.20.0.0 255.255.255.0')])"


,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,before_summary_asa,SPLIT-ACL,Start Location: before_summary_asaSrc IP: 192.168.1.0Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,PERMIT,permit 10.20.0.0 255.255.255.0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by standard ipv4 access-list named SPLIT-ACL, index 0: permit 10.20.0.0 255.255.255.0', lineDescription='permit 10.20.0.0 255.255.255.0')])"


,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,before_summary_asa,SPLIT-ACL,Start Location: before_summary_asaSrc IP: 172.168.1.0Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,PERMIT,permit 10.20.0.0 255.255.255.0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by standard ipv4 access-list named SPLIT-ACL, index 0: permit 10.20.0.0 255.255.255.0', lineDescription='permit 10.20.0.0 255.255.255.0')])"


In [2]:
pip show jupyter

Name: jupyter
Version: 1.0.0
Summary: Jupyter metapackage. Install all the Jupyter components in one go.
Home-page: http://jupyter.org
Author: Jupyter Development Team
Author-email: jupyter@googlegroups.org
License: BSD
Location: /usr/local/lib/python3.7/site-packages
Requires: nbconvert, ipywidgets, qtconsole, notebook, ipykernel, jupyter-console
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
%%which python

UsageError: Cell magic `%%which` not found.


In [9]:
import sys
print(sys.prefix)

/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7


In [10]:
import sys
print(sys.path)

['/Users/lisago/python/project/Batfish/Batfish/notebooks', '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/lisago/Library/Python/3.7/lib/python/site-packages', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/IPython/extensions', '/Users/lisago/.ipython']


In [ ]:
/Users/lisago/python/ansible/venv/lib/python3.7/site-packages

In [16]:
import os
import sys
sys.path.append('/Users/lisago/python/ansible/venv/lib/python3.7/site-packages')

import textfsm

In [11]:
import json
ip_owners_ans = bfq.ipOwners().answer()
#print(ip_owners_ans)
json_frame = ip_owners_ans.frame()
print(json_frame.to_json())

{"Node":{"0":"after_summary_asa","1":"ciscoasa","2":"n9kv-1","3":"n9kv-1","4":"after_summary_asa","5":"before_summary_asa","6":"n9kv-1","7":"n9kv-1","8":"ciscoasa","9":"ciscoasa","10":"n9kv-1","11":"after_summary_asa","12":"before_summary_asa","13":"n9kv-1","14":"n9kv-1","15":"after_summary_asa","16":"before_summary_asa","17":"before_summary_asa","18":"before_summary_asa","19":"before_summary_asa","20":"before_summary_asa","21":"before_summary_asa","22":"ciscoasa","23":"ciscoasa","24":"ciscoasa","25":"ciscoasa","26":"ciscoasa","27":"before_summary_asa","28":"after_summary_asa","29":"after_summary_asa","30":"after_summary_asa","31":"after_summary_asa","32":"after_summary_asa","33":"ciscoasa"},"VRF":{"0":"default","1":"default","2":"default","3":"default","4":"default","5":"default","6":"default","7":"default","8":"default","9":"default","10":"default","11":"default","12":"default","13":"management","14":"default","15":"default","16":"default","17":"default","18":"default","19":"default"

In [12]:
ip_flow = HeaderConstraints(srcIps='192.168.1.2/24')
                             #dstIps='255.255.255.0')
answer = bfq.testFilters(headers=ip_flow,
                             nodes='before_summary_asa',
                             filters="SPLIT-ACL").answer() 
show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,before_summary_asa,SPLIT-ACL,Start Location: before_summary_asaSrc IP: 192.168.1.0Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,PERMIT,permit 10.20.0.0 255.255.255.0,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.PermittedByIpAccessListLine', description='Flow permitted by standard ipv4 access-list named SPLIT-ACL, index 0: permit 10.20.0.0 255.255.255.0', lineDescription='permit 10.20.0.0 255.255.255.0')])"
